In [ ]:
import os
import numpy as np
import warnings
from astropy.coordinates import Angle
from astropy.io import fits
from astropy.modeling import models, fitting
import glob
from barycorrpy import get_BC_vel
from scipy import interpolate
import matplotlib.pyplot as plt
import json
import glob
%matplotlib inline

In [ ]:
fit_g = fitting.LevMarLSQFitter()

In [ ]:
def plot_gaussian_on_curve(g_curve, curve_x, curve_y, order=None, title=None, harps=None, v_loop=None, harps_f=None, first_label=None, second_label=None):
    plt.figure(figsize=(8,8))
    
    label_curve = "ccf of " + str(order) if order is not None else ('all orders from ' + first_label)
    plt.plot(curve_x, curve_y, 'ko', label=label_curve)
    if harps is not None:
        plt.plot(curve_x, harps, 'mo', label="ccf " if second_label is None else ('from '+second_label))
    plt.plot(curve_x, g_curve(curve_x), label='1st mean:'+str("{0:.6f}".format(g_curve.mean.value)))
    if harps_f is not None:
        plt.plot(curve_x, harps_f(curve_x), label = '2nd mean:'+ str("{0:.6f}".format(harps_f.mean.value)))
    plt.legend(loc="lower right", prop={'size': 12})
    plt.title(title) if title is not None else None

In [ ]:
def fit_ccf(velocities, ccf, rv_guess, velocity_cut=100.0):
    g_init = models.Gaussian1D(amplitude=-1e7, mean=rv_guess, stddev=5.0)
    i_cut = (velocities >= rv_guess - velocity_cut) & (velocities <= rv_guess + velocity_cut)
    g_x = velocities[i_cut]
    g_y = ccf[i_cut]-np.nanmedian(ccf)
    gaussian_fit = fit_g(g_init, g_x, g_y)
    rv_result = gaussian_fit.mean.value
    return gaussian_fit, rv_result, g_x, g_y


In [ ]:
def get_harps_ccf(keystr):
    #import pdb; pdb.set_trace()
    harps_file =  '../test_data/rv_test/HARPStauceti_baseline/ccf/HARPS'+keystr+'_ccf_G2_A.fits'
    harps_ccf, ccf_head = fits.getdata(harps_file, header=True)
    return harps_ccf

## Compare Arpita output with reference output

In [ ]:
rv_output_fits = glob.glob('../test_data/rv_test/output_arpita/*.fits')
rv_output_fits.sort()
total_fits = len(rv_output_fits)
print(total_fits)

for n in range(total_fits):
    #import pdb; pdb.set_trace()
    fits_name = rv_output_fits[n]   
    i1 = fits_name.index('ARPITA.20')+6
    harps_ccf = get_harps_ccf(fits_name[i1:i1+24])
    rv_data, head = fits.getdata(fits_name, header=True)
    s = np.shape(rv_data)
    v_size = s[0]
    vel_sum = rv_data[v_size-1, :]
    vel_loop = rv_data[v_size-2, :]
    gaussian_fit, rv_result, g_x, g_y = fit_ccf(vel_loop, vel_sum, vel_loop[int(s[1]/2)])
    harps_fit, harps_mean, h_x, h_y = fit_ccf(vel_loop, harps_ccf[v_size-1], vel_loop[int(s[1]/2)])
    plot_gaussian_on_curve(gaussian_fit, g_x, g_y, title=fits_name[i1:-5]+'-('+str(n)+')', harps_f = harps_fit, harps=h_y, first_label='A', second_label='R')

## compare Cindy's output (codes are ported from Arpita's) with Arpita's output

In [ ]:
a_output_fits = glob.glob('../test_data/rv_test/output_arpita/*.fits')
a_output_fits.sort()
total_fits = len(a_output_fits)
print(total_fits)

c_output_fits = glob.glob('../test_data/rv_test/output_cindy/*.fits')
c_output_fits.sort();

for n in range(total_fits):
    #import pdb; pdb.set_trace()
    fits_name = a_output_fits[n]   
    i1 = fits_name.index('ARPITA.20')+ 6
    #harps_ccf = get_harps_ccf(fits_name[i1:i1+24])
    rv_data, head = fits.getdata(fits_name, header=True)
    c_ccf, c_ccd_h = fits.getdata(c_output_fits[n], header=True)
    s = np.shape(rv_data)
    v_size = s[0]
    vel_sum = rv_data[v_size-1, :]
    vel_loop = rv_data[v_size-2, :]
    gaussian_fit, rv_result, g_x, g_y = fit_ccf(vel_loop, vel_sum, vel_loop[int(s[1]/2)])
    c_fit, c_mean, h_x, h_y = fit_ccf(vel_loop, c_ccf[v_size-1], vel_loop[int(s[1]/2)])
    plot_gaussian_on_curve(gaussian_fit, g_x, g_y, title=fits_name[i1:-5]+'-('+str(n)+')', harps_f = c_fit, harps=h_y, first_label='C', second_label='A')
    

## compare fixed result with original result 

In [ ]:
a_output_fits = glob.glob('../test_data/rv_test/output_cindy_fix/*.fits')
a_output_fits.sort()
total_fits = len(a_output_fits)
print(total_fits)

#c_output_fits = glob.glob('../test_data/rv_test/output_cindy/*.fits')
c_output_fits = glob.glob('../RadialVelocity/output_cindy_1011_fix/*.fits')
c_output_fits.sort();

for n in range(total_fits):
    #import pdb; pdb.set_trace()
    fits_name = a_output_fits[n]   
    i1 = fits_name.index('Cindy.20')+5
    rv_data, head = fits.getdata(fits_name, header=True)
    c_ccf, c_ccd_h = fits.getdata(c_output_fits[n], header=True)
    s = np.shape(rv_data)
    v_size = s[0]
    vel_sum = rv_data[v_size-1, :]
    vel_loop = rv_data[v_size-2, :]
    gaussian_fit, rv_result, g_x, g_y = fit_ccf(vel_loop, vel_sum, vel_loop[int(s[1]/2)])
    c_fit, c_mean, h_x, h_y = fit_ccf(vel_loop, c_ccf[v_size-1], vel_loop[int(s[1]/2)])
    plot_gaussian_on_curve(gaussian_fit, g_x, g_y, title=fits_name[i1:-5]+'-('+str(n)+')', harps_f = c_fit, harps=h_y, first_label='C1', second_label='C2')
    